# **1. Clone repository**
---
Cloning the repository will give you access to the model and scripts we will use for denoising. After the operation is finished, you will find the files and models in the **Files** tab on the top left of notebook.

In [ ]:
!git clone https://github.com/LuisFMCuriel/MINDLAB-Denoising-bioluminescence-images.git

# **1.2. Install CSBDeep for CARE and import dependencies**
---
By default a few libriaries are not installed in the google colab environment. Running this cell will ensure the libraries we need for denoising are fully operational and imported

In [ ]:
#@markdown ##Install CARE and dependencies

# Here, we install libraries which are not already included in Colab.
!pip install csbdeep


import sys
import tensorflow 
import tensorflow as tf


print(tensorflow.__version__)
print("Tensorflow enabled.")


from csbdeep.utils import download_and_extract_zip_file, plot_some, axes_dict, plot_history, Path, download_and_extract_zip_file
from csbdeep.data import RawData, create_patches 
from csbdeep.io import load_training_data, save_tiff_imagej_compatible
from csbdeep.models import Config, CARE
from csbdeep import data
%matplotlib inline


import numpy as np
from matplotlib import pyplot as plt
import os
import shutil 
from tifffile import imread, imsave
import subprocess

## **2.1. Check for GPU access**
---

By default, the session should be using Python 3 and GPU acceleration, but it is possible to ensure that these are set properly by doing the following:

<font size = 4>Go to **Runtime -> Change the Runtime type**

In [ ]:
if tf.test.gpu_device_name()=='':
        print('You do not have GPU access.') 

else:
        print('You have GPU access')
        subprocess.run(["nvidia-smi"])

## **2.2. Mount your Google Drive**
---
In case you want to use the data you acquired during the workshop, you need to mount your Google Drive to this notebook.

Play the cell below to mount your Google Drive and follow the link. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive. 

Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:

#@markdown ##Run this cell to connect your Google Drive to Colab

#@markdown * Click on the URL. 

#@markdown * Follow the instructions

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive". 

#mounts user's Google Drive to Google Colab.

from google.colab import drive
drive.mount('/content/gdrive')

## **2.3. Select the inference folder**
---
Select the **Test_data_folder** (where the noisy images are) and the **Result_folder_root** (where the denoised inferences are going to be saved).

In [ ]:
#@markdown ###Path to denoise images:

Test_data_folder = "" #@param {type:"string"}

Result_folder_root = "" #@param {type:"string"}


## **3.0. Perform denoising**
---
Using the pretrained model in **Your_Github_username/MINDLAB-denoising-bioluminescence-images/model** perform the denoising in the images stored in the path selected before.

In [ ]:
inference_model_name = "denoising_model" #@param {type:"string"}
inference_model_path = r"/content/MINDLAB-Denoising-bioluminescence-images/model"

Result_folder = Result_folder_root

# Create directories for the results
if not os.path.exists(Result_folder):
    os.makedirs(Result_folder)
else:
    shutil.rmtree(Result_folder)
    os.makedirs(Result_folder)


#Activate the pretrained model. 
model_training = CARE(config=None, name=inference_model_name, basedir=inference_model_path)

STACK = "N"
# creates a loop, creating filenames and saving them
if STACK == "Y":
    for filename in os.listdir(Test_data_folder):
        img = imread(os.path.join(Test_data_folder,filename))
        r = np.zeros_like(img)
        for s in range(img.shape[0]):
            restored = model_training.predict(img[s,:,:], axes='YX')
            r[s,:,:] = restored
        os.chdir(Result_folder)
        imsave(filename,r.astype("uint16"))
else:

    for filename in os.listdir(Test_data_folder):
        img = imread(os.path.join(Test_data_folder,filename))
        restored = model_training.predict(img, axes='YX')
        os.chdir(Result_folder)
        imsave(filename,restored)


print("Images saved into folder:", Result_folder)